In [1]:
import os
import random
import json
import hashlib
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
!pwd

/home/user/workspace/itmo_courses/разные проекты


In [3]:
strory = '''
The story begins with two unnamed officers of unknown rank sitting in the guardroom for unspecified crimes. Due to boredom, one of the officers, subsequently referred to as "Crazy," starts telling his life stories to the other officer, referred to as "Brother." Throughout his narrative, the level of madness in Crazy's tales escalates. He recounts incidents such as an attempt to humiliate a fellow soldier by defecating in his boot during mandatory service, consuming feces, and his first sexual experience ("Had a few shots of 'Triple Seven' with some crazy chick, and then we messed around").
Initially, Brother finds these stories amusing, but in the confined space with such an "interesting" cellmate, he becomes increasingly uncomfortable. He asks Crazy to "shut up," but Crazy refuses. In response, Brother reports him to the authorities. Later, Crazy notices a lot of flies in their cell. In an attempt to rid Brother of the flies, Crazy suggests defecating on the floor, hoping the flies will gather, and he can kill them. However, Brother beats him again.
Crazy groans in pain, and the noise attracts the attention of the Guard, a vile and weak man with authority over cellmates, who takes Brother away for "work." The so-called "work" involves cleaning a dirty toilet using a dining fork. The Guard explicitly instructs Brother to "clean the crap." Brother, either genuinely not understanding the task or pretending to be repulsed, sits and curses the guard. The Guard returns to the toilet, efficiently, if not "virtuosically," scraping it with the fork to demonstrate the proper technique. However, Brother stubbornly refuses to comply, openly declaring to the returning Guard that he won't clean. Furious, the Guard drives Brother out of the latrine.
When Brother returns to the cell, Crazy decides to sing him a song about the "green elephant," a nonsensical and childlike tune inappropriate for their setting. After their conversation, Brother goes to sleep, while Crazy prepares a "breakfast" for his friend – defecating in a bowl, smearing feces on the floor and his belly, and even sniffing and eating it. The next morning, Crazy brings Brother a so-called "sweet bread" (referring to his own excrement) in the only clean bowl in the cell.
This becomes the last straw for Brother, who, with loud cries, pleads to "take this shit-eater away." However, he himself goes back to clean the toilet. While Brother learns the basics of this "art," Captain, the head of the guardroom, enters Crazy's cell and starts narrating a story, heavily seasoned with profanity (and factual errors), about the Japanese attack on Pearl Harbor. Although Crazy struggles to comprehend this "classic" tale, the Captain imparts his intended message. After the lecture, the Guard, following the Captain's orders, beats Crazy. For many readers of 2ch and YouTube, the story concludes here, as the subsequent events are too explicit and bloody to describe. Nevertheless, the story must be completed.
Following further Captain-induced torment in the "hole," Brother loses his sanity entirely, kills the Captain, and then slashes his own wrists. The Guard, now donning the Captain's uniform, believes himself to be a colonel and aspires to "go to the Officers' Club" but ultimately decides to hang himself as well. Meanwhile, Crazy remains alone in the "hole," singing about the "Green Elephant" amidst intestines and blood. Instead of Brother's corpse, he hallucinates his own mother, engaging in a conversation with her.
'''

In [4]:
#validation_data, test_data = load_dataset("suolyer/pile_wikipedia", split=['validation', 'test'])

data = []
#random_rows = random.sample(range(len(test_data)), 10)
build_data = toker = strory.split('.')

m = hashlib.md5()
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def bert_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

def create_chunk_dataset(content):
    m.update(content.encode('utf-8'))
    uid = m.hexdigest()[:12]
    text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 400,
          chunk_overlap  = 40,
          length_function = bert_len,
          separators=['\n\n', '\n', ' ', ''],
      )
    chunks = text_splitter.split_text(content)
    for i, chunk in enumerate(chunks):
          data.append({
              'id': f'{uid}-{i}',
              'text': chunk
          })

for dt in build_data:
    create_chunk_dataset(dt)

filename = 'rebel_llamaindex/green_elephant_chunks.jsonl'
# save
with open(filename, 'w') as outfile:
    for x in data:
        outfile.write(json.dumps(x) + '\n')

In [5]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

In [6]:
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 1,
}

triples = []

def generate_triples(texts):
    model_inputs = tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    generated_tokens = model.generate(
      model_inputs["input_ids"].to(model.device),
      attention_mask=model_inputs["attention_mask"].to(model.device),
      **gen_kwargs
   )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    for idx, sentence in enumerate(decoded_preds):
        et = extract_triplets(sentence)
        for t in et:
            triples.append((t['head'], t['type'], t['tail']))

for i in tqdm(range(0, len(data), 2)):
    try:
        texts = [data[i]['text'], data[i+1]['text']]
    except:
        texts = [data[i]['text']]
    generate_triples(texts)

distinct_triples = list(set(triples))

# save
with open('rebel_triples.json', 'w') as file:
    json.dump(distinct_triples, file)

  0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
distinct_triples[:3]

[('Captain', 'characters', 'Crazy'),
 ('2ch', 'different from', 'YouTube'),
 ('intestines', 'has part', 'blood')]

In [8]:
len(distinct_triples)

35

In [9]:
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,)

In [11]:
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = 'your_key'

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  

from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import OpenAI

from IPython.display import Markdown, display

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [ ]:
from llama_index.graph_stores import SimpleGraphStore
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import SimpleGraphStore

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)
documents = SimpleDirectoryReader("rebel_llamaindex/").load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

In [16]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

In [22]:
space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], ["relationship"] 
tags = ["entity"]

In [28]:
rebel_kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    kg_triplet_extract_fn=extract_triplets,
    storage_context=storage_context,
    max_triplets_per_chunk=5,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)